In [43]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error
import networkx as nx
import math
import sys
import pandas as pd
import numpy as np

In [44]:
df = pd.read_csv('./OTCNet.csv')
X =  df
#y =  df['rating']
X = X.values
#y = y.values
loo = LeaveOneOut()
loo.get_n_splits(X)

35592

In [45]:
def initialize_scores(G):
    fairness = {}
    goodness = {}
    
    nodes = G.nodes()
    #print(nodes)
    for node in nodes:
        fairness[node] = 1
        try:
            goodness[node] = G.in_degree(node, weight='weight')*1.0/G.in_degree(node)
        except:
            goodness[node] = 0
    return fairness, goodness

In [46]:
def compute_fairness_goodness(G):
    fairness, goodness = initialize_scores(G)
    
    nodes = G.nodes()
    iter = 0
    while iter < 100:
        df = 0
        dg = 0
        for node in nodes:
            inedges = G.in_edges(node, data='weight')
            g = 0
            for edge in inedges:
                g += fairness[edge[0]]*edge[2]

            try:
                dg += abs(g/len(inedges) - goodness[node])
                goodness[node] = g/len(inedges)
            except:
                pass
            
        for node in nodes:
            outedges = G.out_edges(node, data='weight')
            f = 0
            for edge in outedges:
                f += 1.0 - abs(edge[2] - goodness[edge[1]])/2.0
            try:
                df += abs(f/len(outedges) - fairness[node])
                fairness[node] = f/len(outedges)
            except:
                pass
        
        if df < math.pow(10, -6) and dg < math.pow(10, -6):
            break
        iter+=1
    
    return fairness, goodness

In [47]:
fairness = {}
goodness = {}
fairness_test = {}
goodness_test = {}
itr = 10
for train_index, test_index in loo.split(X):
    if test_index[0] < (itr):
        X_train, X_test = X[train_index], X[test_index]
        G = nx.DiGraph()
        for index in train_index:
            G.add_edge(X_train[index-1][0], X_train[index-1][1], weight = float(X_train[index-1][2])) ## the weight should already be in the range of -1 to 1
        fairness, goodness = compute_fairness_goodness(G)
        try:
            fairness_test[test_index[0]] = fairness[X_test[0][0]]
        except:
            fairness_test[test_index[0]] = 0
            pass
        try:
            goodness_test[test_index[0]] = goodness[X_test[0][1]]
        except:
            goodness_test[test_index[0]] = 0
            pass
    else:
        break
    print('iteration', test_index[0])

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [48]:
df['fairness'] = pd.Series(fairness_test, index=df.index, dtype=float)
df['goodness'] = pd.Series(goodness_test, index=df.index, dtype=float)

In [49]:
df = df.dropna()
df.to_csv('./leaveoneout_fairness_goodness.csv')

In [50]:
RMSE_fg_leave = 0
RMSE_fg_leave += math.sqrt(mean_squared_error(df['rating'], df['fairness']*df['goodness']))

In [51]:
RMSE_fg_leave

0.4095057156102173

In [52]:
# Pearson correlation coefficient
np.corrcoef(df['rating'], df['fairness']*df['goodness'])

array([[ 1.        , -0.12052684],
       [-0.12052684,  1.        ]])